# 3. Modelagem Para Previsão

## 3.1: Carregando e limpando os dados

In [247]:
# Importanto componentes para o aprendizado de máquina
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score

In [248]:
# Carregando a base de dados
df_precificacao = pd.read_csv('teste_indicium_precificacao.csv')

In [249]:
# Verificando quantidade de valores faltantes em cada coluna
df_precificacao.isnull().sum()

# Removendo colunar com mais de 30% de dados faltando
limite = len(df_precificacao) * 0.7
df_precificacao = df_precificacao.dropna(thresh=limite, axis=1)

# Removendo duplicatas baseadas no id
df_precificacao = df_precificacao.drop_duplicates(subset=['id'])

## 3.2: Pré-processamento

In [251]:
# Excluindo colunas que não serão úteis
df_modelo = df_precificacao.drop(['id', 'nome', 'host_id', 'host_name', 'ultima_review'], axis=1)

# Substiui valores NAN por zero
df_modelo['reviews_por_mes'] = df_modelo['reviews_por_mes'].fillna(0)

In [252]:
# Codificando variáveis
categoricas = ['bairro_group', 'bairro', 'room_type']
numericas = ['latitude', 'longitude', 'minimo_noites', 'numero_de_reviews', 'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']

In [253]:
# Aplicando as transformações nas colunas
preprocessador = ColumnTransformer(transformers=[('num', StandardScaler(), numericas), ('cat', OneHotEncoder(handle_unknown='ignore'), categoricas)])

## 3.3: Treinando o modelo (Regressão)

In [255]:
# Definindo X e y
X = preprocessador.fit_transform(df_modelo.drop('price', axis=1))
y = df_modelo['price']

# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando o modelo
modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Realizando a avaliação
y_pred = modelo.predict(X_test)
EQM = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Erro Quadrático Médio (EQM): {EQM:.2f}')
print(f'R²: {r2:.8f}')

Erro Quadrático Médio (EQM): 214.29
R²: 0.00407520


# 4. Prevendo Preço Do Apartamento

In [257]:
# Dados do apartamento
apartamento = {
 'bairro_group': 'Manhattan',
 'bairro': 'Midtown',
 'room_type': 'Entire home/apt',
 'latitude': 40.75362,
 'longitude': -73.98377,
 'minimo_noites': 1,
 'numero_de_reviews': 45,
 'reviews_por_mes': 0.38,
 'calculado_host_listings_count': 2,
 'disponibilidade_365': 355}

# Criando um DataFrame
df_apartamento = pd.DataFrame([apartamento])

# Pré-processando
X_apartamento = preprocessador.transform(df_apartamento)

# Prevendo
preco = modelo.predict(X_apartamento)[0]
print(f'Sugestão de preço: US$ {preco:.2f}')

Sugestão de preço: US$ 477.84
